# Introduction to Transform objects

This Jupyter notebook includes an introduction to Transform objects, and worked examples below.

Transform is a general-purpose framework to represent, manipulate, and use coordinate transformations on scientific data.  A Transform object represents a coordinate transformation mapping $ ℝ^{N}→ ℝ^{M} $ , for integers $ N $ and $ M $.  Subclasses of Transform represent families of transformation, which can be parameterized at construction time. Instances of Transform represent individual coordinate transformations, which are fixed but can be composed, inverted, applied, or used to resample or remap array data such as images.

The transform core module includes a WCS object that allows you to work with World Coordinate System transformations mapping pixel index values in an image to/from scientific coordinates associated with that image.  WCS is a standard for metadata in scientific imaging, and is used throughout the astronomy and heliophysics scientific communities, but it is sufficiently general for other applications as well.

This notebook includes demonstrations of basic Transform construction and manipulation, and shows how Transforms can be applied to image data to remap/resample them.

## Basics

### Manipulating Vectors

In general, Transforms apply to vector data.  Most Transforms supply both forward and reverse operations.  Some transformations are not reversible, so it is possible for a Transform to have only a forward operation or only a reverse operation.

#### Vectors and broadcasting

Vectors are Python "list-like objects" (typically NumPy arrays) that you supply with indices in conventional order (X,Y,...).  Because of the way NumPy broadcast rules work, by convention Transform objects expect the vector index to run across the *last* dimension of a supplied NumPy array.  So in general, a coordinate transform that expects to operate on 2-vectors can operate in a (heh) vectorized manner on any NumPy array whose last dimension has a size of 2 or more. If you supply the Transform with a NumPy array whose shape is
`[1000,1000,2]`, you'll get the same operation applied a million times -- once per 2-vector in the source array.  

#### Dimensionality of a transform

Individual Transform subclasses or instances can define the dimensionality of the vectors they expect to 
receive and/or generate.  These are carried in the `idim` and `odim` attributes, which may contain 0 to indicate that the Transform can operate on vectors of any size, or a positive integer to indicate the 
dimensionality of the vector.  Transforms whose `odim` is lower than the `idim` are *projections* and reduce the dimensionality of their input vectors.  Transforms whose `odim` is higher than the `idim` are *embeddings* and increase the dimensionality of their input vectors. 

In general, if you feed in a higher dimensional vector than a transform expects, the additional dimensions are preserved untouched -- so if you have a coordinate transformation that expects a 2-D input vector and projects it to 1-D, then a 3-D input vector will be projected to 2-D (and the additional components are preserved untouched at the end of the manipulated components of the vector).

Given a Transform object (see below for examples), you can us it to manipulate vector data using the `apply` method.  Apply accepts one or more vectors as a list-like object, and returns the transformed vectors.

### Resampling and Remapping Arrays

Transforms can also be used to remap array data, by manipulating the pixel coordinate system in the data.

#### Pixel coordinate system; NumPy reverse indexing

Array data such as images have an implicit coordinate system.  Each pixel of an image is associated with a 2-vector indicating its location in the pixel grid.  Following NumPy convention, indexing is in *reverse* order: while the vector data manipulated by `apply` is in (X,Y) order, images and other data are considered to be indexed in (Y,X) order, with X increasing to the right and Y increasing upward.  

#### Resampling in the pixel coordinate system 

You can use Transform objects to change the shape of data within their pixel grid: notionally, the coordinate transformation is applied to each pixel's coordinate vector, and the data value moves to the new location in the pixel grid.  This resampling is implemented via the inverse of the Transform: every pixel grid location in the output image is mapped backward to a location on the input grid, and a data value is interpolated from the input at that location.  Resampling is implemented with the `resample` method (below).  

#### Remapping in a scientific coordinate system

You can also use transform objects to manipulate the scientific coordinates of data in an image, using the framework of Astropy support for the World Coordinate System standard.  This adds one level of indirection to resampling: you can distort, transform, tweak, or scale images in their original scientific coordinate system rather than in the implicit pixel grid coordinate system.  Remapping in this way is implemented with the `remap` method.

## Transform API methods

Transforms tightly encapsulate the messiness of general coordinate transformation.  Each Transform object has only a few methods:

- **_Constructor_**: Each Transform subclass has a constructor.  You should always construct a subclass - Transform itself is only a container/superclass.

- **`apply`**: This applies the Transform to vector data, returning a modified version of the input vectors.  In addition to the Transform itself, you supply the vector data and an optional `invert` flag indicating whether the forward or reverse transformation is to be applied.

- **`invert`**: This applies the inverse of the Transform to vector data.  It's equivalent to setting the `invert` flag in `apply()`.

- **`inverse`**: This returns a different Transform that implements the inverse of the original.

- **`resample`**: This uses the Transform to resample a NumPy array or similar object in its intrinsic pixel coordinate system.  There are several options that control the shape of the output array, the interpolation method, and any boundary conditions to apply.

- **`remap`** This uses the Transform to resample a NumPy array or similar object in an associated scientific coordinate system.  It makes use of a FITS header or astropy.wcs object to associate a scientific coordinate system with the array.  You can supply the data in any one of several common formats:  a tuple containing (data, header); a dictionary containing data and header fields; or a generic object containing data and header attributes.  Remap will try to give you back the data in the same format you supplied it, if possible.  Remap will autoscale the output so that the input range fits in the output pixel plane, and return a WCS header that reflects the scaling.  There are several options controlling the shape and scaling of the output array, the interpolation method, and boundary conditions.





## Examples 

### Loading Transform

Transform works closely with numpy and with FITS objects, so you will want to import those modules up-front for convenience.

In [2]:
import numpy as np
import astropy.io.fits
import transform as t